### EXAMEN SCALE
#### Etudiants:
    - El Hadji Bassirou DEME 217897
    - Oumar Alpha Yaya CISSE 218015
    - Mouhamadou Makhtar LY 219287

### Lecture de la data

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.5`
import $ivy.`sh.almond::almond-spark:0.10.9`

import $ivy.$                                  

import $ivy.$                               

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.log4j.{Level, Logger}


In [7]:
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._

import org.apache.spark.sql.{DataFrame, SparkSession}

import org.apache.spark.sql.functions._

import org.apache.spark.sql.SparkSession

import org.apache.spark.sql.functions._

In [3]:
import org.apache.spark.sql._

val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@731aa97d

In [8]:
import spark.implicits._

import spark.implicits._

### Importation des données

In [5]:
val path = "archive/*"
val df = spark
        .read
        .option("header","true")
        .csv(path)

csv at cmd4.sc:5

1 / 1

path: String = "archive/*"
df: DataFrame = [Order ID: string, Product: string ... 4 more fields]

## Chargement des données

In [6]:
df.show()

show at cmd5.sc:1

1 / 1

+--------+--------------------+----------------+----------+--------------+--------------------+
|Order ID|             Product|Quantity Ordered|Price Each|    Order Date|    Purchase Address|
+--------+--------------------+----------------+----------+--------------+--------------------+
|  295665|  Macbook Pro Laptop|               1|      1700|12/30/19 00:01|136 Church St, Ne...|
|  295666|  LG Washing Machine|               1|     600.0|12/29/19 07:03|562 2nd St, New Y...|
|  295667|USB-C Charging Cable|               1|     11.95|12/12/19 18:21|277 Main St, New ...|
|  295668|    27in FHD Monitor|               1|    149.99|12/22/19 15:13|410 6th St, San F...|
|  295669|USB-C Charging Cable|               1|     11.95|12/18/19 12:38|43 Hill St, Atlan...|
|  295670|AA Batteries (4-p...|               1|      3.84|12/31/19 22:58|200 Jefferson St,...|
|  295671|USB-C Charging Cable|               1|     11.95|12/16/19 15:10|928 12th St, Port...|
|  295672|USB-C Charging Cable|         

#### Encodage de la variable date

In [9]:
import org.apache.spark.sql.functions.to_timestamp

val dfTimestamp = df.withColumn("Order Date", to_timestamp(col("Order Date"), "MM/dd/yyyy HH:mm"))

import org.apache.spark.sql.functions.to_timestamp


dfTimestamp: DataFrame = [Order ID: string, Product: string ... 4 more fields]

1. Quelle année a été la meilleure en termes de ventes ? Combien a-t-on gagné cette
année-là ?

In [10]:
val dfYear = dfTimestamp
  .groupBy(year(col("Order Date")).as("Annee"))
  .agg(sum(col("Quantity Ordered") * col("Price Each")).as("Total"))
  .orderBy(desc("Total"))

  
val bestYear = dfYear.first().getAs[Int]("Annee")
val total = dfYear.first().getAs[Double]("Total")

println(s"La meilleur annee est 20$bestYear avec un gain total de $total en terme de ventes.")

first at cmd9.sc:7

6 / 6

first at cmd9.sc:7

200 / 200

first at cmd9.sc:8

6 / 6

first at cmd9.sc:8

200 / 200

La meilleur annee est 2019 avec un gain total de 3.379903641001321E7 en terme de ventes.


dfYear: Dataset[Row] = [Annee: int, Total: double]
bestYear: Int = 19
total: Double = 3.379903641001321E7

2. Quel mois a été le meilleur en termes de ventes ? Combien a-t-on gagné ce mois-là ?

In [14]:
val MTH = List("Janvier", "Février", "Mars", "Avril", "Mai", "Juin", "Juillet", "Août", "Septembre", "Octobre", "Novembre", "Décembre")

MTH: List[String] = List(
  "Janvier",
  "F\u00e9vrier",
  "Mars",
  "Avril",
  "Mai",
  "Juin",
  "Juillet",
  "Ao\u00fbt",
  "Septembre",
  "Octobre",
  "Novembre",
  "D\u00e9cembre"
)

In [19]:
val dfMonth = dfTimestamp
  .groupBy(month(col("Order Date")).alias("Mois"))
  .agg(sum(col("Quantity Ordered") * col("Price Each")).as("Total"))
  .orderBy(desc("Total"))

val bestMonth = dfMonth.first().getAs[Int]("Mois")
val total = dfMonth.first().getAs[Double]("Total")

val m_month = MTH(bestMonth-1)
println(s"Le meilleur mois est $m_month avec un gain de $total en terme de vente")

first at cmd18.sc:6

6 / 6

first at cmd18.sc:6

200 / 200

first at cmd18.sc:7

6 / 6

first at cmd18.sc:7

200 / 200

Le meilleur mois est Novembre avec un gain de 3715537.459999819 en terme de vente


dfMonth: Dataset[Row] = [Mois: int, Total: double]
bestMonth: Int = 11
total: Double = 3715537.459999819
m_month: String = "Novembre"

- Le mois qui a été le meilleur en terme de vente est le mois de decembre, avec un gain de 2780850

3. Quelle ville a enregistré le plus grand nombre de ventes ?

In [20]:
val dfCity = df.withColumn("City", split(col("Purchase Address"), ",")(1))

val dfVenteByCity = dfCity
  .groupBy("City")
  .agg(count("City").alias("Total"))
  .orderBy(desc("Total"))

val bestCity = dfVenteByCity.first().getAs[String]("City")

println(s"La ville ayant enregistré le plus grand nombre de vente est $bestCity")

first at cmd19.sc:8

6 / 6

first at cmd19.sc:8

200 / 200

La ville ayant enregistré le plus grand nombre de vente est  San Francisco


dfCity: DataFrame = [Order ID: string, Product: string ... 5 more fields]
dfVenteByCity: Dataset[Row] = [City: string, Total: bigint]
bestCity: String = " San Francisco"

4. À quelle heure devraient-ils diffuser des publicités pour maximiser les chances que les
clients achètent le produit ?

In [22]:
val DFhour = dfTimestamp.select(hour($"Order Date").alias("Hour"))

val hourBest = DFhour.groupBy("Hour").count().orderBy(desc("count")).first().getAs[Int]("Hour")

println(s"Pour maximiser les chances l'heure a la quelle il doit diffuser les publicités est: $hourBest heure.")

first at cmd21.sc:3

6 / 6

first at cmd21.sc:3

200 / 200

Pour maximiser les chances l'heure a la quelle il doit diffuser les publicités est: 19 heure.


DFhour: DataFrame = [Hour: int]
hourBest: Int = 19

5. Quels produits sont le plus souvent vendus ensemble ?

In [24]:
val df1 = dfTimestamp.groupBy("Order ID")
  .agg(collect_set("Product").as("Products"))

val Pairs = df1.flatMap { row =>
  val products = row.getAs[Seq[String]]("Products")
  products.combinations(2).map(pair => (pair(0), pair(1)))
}.toDF("Prod 1", "Prod 2")

val PairCounts = Pairs.groupBy("Prod 1", "Prod 2")
  .count()
  .withColumn("Occurrences", col("count").cast("Int"))
  .drop("count")

val resultat = PairCounts.orderBy(col("Occurrences").desc)

resultat.show(truncate=false)

show at cmd23.sc:16

6 / 6

show at cmd23.sc:16

200 / 200

show at cmd23.sc:16

200 / 200

+------------------------+--------------------------+-----------+
|Prod 1                  |Prod 2                    |Occurrences|
+------------------------+--------------------------+-----------+
|Lightning Charging Cable|iPhone                    |1011       |
|USB-C Charging Cable    |Google Phone              |997        |
|iPhone                  |Wired Headphones          |462        |
|Wired Headphones        |Google Phone              |422        |
|Apple Airpods Headphones|iPhone                    |373        |
|Vareebadd Phone         |USB-C Charging Cable      |368        |
|Google Phone            |Bose SoundSport Headphones|228        |
|USB-C Charging Cable    |Wired Headphones          |203        |
|Vareebadd Phone         |Wired Headphones          |149        |
|Lightning Charging Cable|Wired Headphones          |129        |
|Lightning Charging Cable|AA Batteries (4-pack)     |106        |
|USB-C Charging Cable    |Bose SoundSport Headphones|102        |
|Apple Air

df1: DataFrame = [Order ID: string, Products: array<string>]
Pairs: DataFrame = [Prod 1: string, Prod 2: string]
PairCounts: DataFrame = [Prod 1: string, Prod 2: string ... 1 more field]
resultat: Dataset[Row] = [Prod 1: string, Prod 2: string ... 1 more field]

6. Quel produit s'est le plus vendu ? Pourquoi pensez-vous qu'il se soit autant vendu ?

In [25]:
val Sells = dfTimestamp.groupBy("Product")
  .agg(sum("Quantity Ordered").as("Total"))
  .orderBy(col("Total").desc)

val bestItems = Sells.first().getAs[String]("Product")

println(s"Le produit le plus vendu est : $bestItems")

first at cmd24.sc:5

6 / 6

first at cmd24.sc:5

200 / 200

Le produit le plus vendu est : AAA Batteries (4-pack)


Sells: Dataset[Row] = [Product: string, Total: double]
bestItems: String = "AAA Batteries (4-pack)"

- Elle sont autant vendu parcequ'elle ont une qualitité plus durable

7. Quelle est la probabilité que les prochains clients commandent un câble de chargement
USB-C ?

In [26]:
val totalOrders = dfTimestamp.select("Order ID").distinct().count()
val usbCOrders = dfTimestamp.filter(col("Product").contains("USB-C Charging Cable")).count()
val usbCProbability = usbCOrders.toDouble / totalOrders.toDouble

println(s"La probabilité que les prochains clients commandent un câble de chargement USB-C est de $usbCProbability")

count at cmd25.sc:1

6 / 6

count at cmd25.sc:1

200 / 200

count at cmd25.sc:1

...

6 / 6

count at cmd25.sc:2

1 / 1

La probabilité que les prochains clients commandent un câble de chargement USB-C est de 0.1227478297905727


totalOrders: Long = 178439L
usbCOrders: Long = 21903L
usbCProbability: Double = 0.1227478297905727

8. Quelle est la probabilité que les prochains clients commandent un iPhone ?

In [27]:
val totalOrders = dfTimestamp.select("Order ID").distinct().count()
val iPhoneOrders = dfTimestamp.filter(col("Product").contains("iPhone")).count()
val iPhoneProbability = iPhoneOrders.toDouble / totalOrders.toDouble

println(s"La probabilité que les prochains clients commandent un iPhone est de $iPhoneProbability")

count at cmd26.sc:1

6 / 6

count at cmd26.sc:1

200 / 200

count at cmd26.sc:1

...

6 / 6

count at cmd26.sc:2

1 / 1

La probabilité que les prochains clients commandent un iPhone est de 0.03834363564019076


totalOrders: Long = 178439L
iPhoneOrders: Long = 6842L
iPhoneProbability: Double = 0.03834363564019076

9. Quelle est la probabilité que les prochains clients commandent un téléphone Google ?

In [28]:
val totalOrders = dfTimestamp.select("Order ID").distinct().count()
val googlePhoneOrders = dfTimestamp.filter(col("Product").contains("Google Phone")).count()
val googlePhoneProbability = googlePhoneOrders.toDouble / totalOrders.toDouble

println(s"La probabilité que les prochains clients commandent un téléphone Google est de $googlePhoneProbability")

count at cmd27.sc:1

6 / 6

count at cmd27.sc:1

200 / 200

count at cmd27.sc:1

...

6 / 6

count at cmd27.sc:2

1 / 1

La probabilité que les prochains clients commandent un téléphone Google est de 0.030962962132717622


totalOrders: Long = 178439L
googlePhoneOrders: Long = 5525L
googlePhoneProbability: Double = 0.030962962132717622

10. Quelle est la probabilité que d'autres personnes commandent des écouteurs filaires ?

In [29]:
val totalOrders = dfTimestamp.select("Order ID").distinct().count()
val wiredHeadphoneOrders = dfTimestamp.filter(col("Product").contains("Wired Headphones")).count()
val wiredHeadphoneProbability = wiredHeadphoneOrders.toDouble / totalOrders.toDouble

println(s"La probabilité que les prochains clients commandent des écouteurs filaires est de $wiredHeadphoneProbability")

count at cmd28.sc:1

6 / 6

count at cmd28.sc:1

200 / 200

count at cmd28.sc:1

...

6 / 6

count at cmd28.sc:2

1 / 1

La probabilité que les prochains clients commandent des écouteurs filaires est de 0.10581767438732564


totalOrders: Long = 178439L
wiredHeadphoneOrders: Long = 18882L
wiredHeadphoneProbability: Double = 0.10581767438732564

#### PRATIQUE

Proposer un cas d'utilisation concret de Machine Learning en utilisant cet ensemble de
données de ventes. Vous devrez identifier un problème spécifique et proposer un modèle
prédictif ou analytique qui peut être utilisé pour résoudre le problème.



    Chargement des données : Charger l'ensemble de données de ventes dans un DataFrame Spark.

    Prétraitement des données : Effectuer des opérations de nettoyage et de transformation sur les données, telles que la suppression des valeurs manquantes, la conversion des types de données appropriés, etc.

    Extraction des fonctionnalités : Sélectionner les fonctionnalités pertinentes pour la prédiction des ventes futures, telles que le produit, la quantité commandée, le prix, la date de commande, etc. Convertir ces fonctionnalités en une représentation adaptée à l'apprentissage automatique, par exemple en utilisant des codifications à chaud pour les variables catégorielles.

    Division des données : Diviser l'ensemble de données en ensembles d'apprentissage et de test pour évaluer les performances du modèle.

    Modélisation : Entraîner un modèle de régression, tel que la régression linéaire, en utilisant les données d'apprentissage. Le modèle prendra en entrée les fonctionnalités sélectionnées et prédira les ventes futures.

    Évaluation du modèle : Évaluer les performances du modèle en utilisant les données de test. Mesurer les métriques d'évaluation telles que l'erreur quadratique moyenne (RMSE) ou le coefficient de détermination (R²) pour évaluer la précision du modèle.

    Utilisation du modèle pour les prévisions : Une fois que le modèle est entraîné et évalué, il peut être utilisé pour prédire les ventes futures pour de nouvelles données ou des périodes futures. Les fonctionnalités pertinentes de ces nouvelles données doivent être extraites et utilisées comme entrée pour le modèle entraîné.

En résumé, en utilisant Spark avec Scala, vous pouvez construire un modèle de régression basé sur les données historiques de ventes pour prédire les ventes futures d'un produit spécifique. Cela permettrait d'aider les entreprises à prendre des décisions éclairées sur la gestion des stocks, la planification de la production, la prévision des revenus, etc.